# Download data

Disini kita tidak bisa langsung membaca file `.parquet` dengan menggunakan pyarrow karena dari librarynya sendiri belum bisa membaca langsung. \
Alih-alih kita download terlebih dahulu, kemudian baru membacanya

In [2]:
import os
import pyarrow.parquet as pq

# URL dan direktori file
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet"
file_dir = "./data/" + url.split("/")[-1].replace('-', '_')

# Pastikan folder 'data' ada
os.makedirs("./data", exist_ok=True)

# Periksa apakah file sudah ada
if os.path.exists(file_dir):
    df = pq.read_table(file_dir).to_pandas()
    display(df.head())
else:
    print(f"Mengunduh file dari {url}")
    os.system(f"wget {url} -O {file_dir}")
    print("Unduhan selesai.")

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-05-01 00:59:15,2024-05-01 01:23:50,1.0,6.10,1.0,N,138,145,1,28.2,7.75,0.5,5.00,0.00,1.0,42.45,0.0,1.75
1,2,2024-04-30 23:58:26,2024-05-01 00:29:42,1.0,11.23,1.0,N,138,249,1,46.4,6.00,0.5,8.72,0.00,1.0,66.87,2.5,1.75
2,2,2024-05-01 00:57:17,2024-05-01 01:14:15,1.0,9.02,1.0,N,138,170,1,35.9,6.00,0.5,10.57,6.94,1.0,65.16,2.5,1.75
3,2,2024-05-01 00:24:47,2024-05-01 00:48:51,1.0,6.53,1.0,N,87,133,1,30.3,1.00,0.5,7.06,0.00,1.0,42.36,2.5,0.00
4,2,2024-05-01 00:11:20,2024-05-01 00:52:10,1.0,14.38,1.0,N,161,165,1,61.8,1.00,0.5,0.00,0.00,1.0,66.80,2.5,0.00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723833 entries, 0 to 3723832
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [4]:
# cek duplicate values
duplicate = df[df.duplicated()]
duplicate

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee


# Inspect data

Kalau diperhatikan, ini adalah sebuah dataset mengenai Taxi (Uber data). \
Terdapat beberapa kejanggalan, yaitu: 
- Passenger count -> 0
- Trip distance -> 0
- Terdapat beberapa baris yang mengandung NaN

Decisions: 
- Menghilangkan data atau baris, dan menggunakan dataframe yang tidak mengandung kriteria di atas

In [5]:
passenger_is_0 = df[(df['passenger_count'].isin([0])) | (df['passenger_count'].isna())]
display(passenger_is_0)

trip_is_0 = df[df['trip_distance'].isin([0])]
display(trip_is_0)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
21,1,2024-05-01 00:23:50,2024-05-01 00:38:22,0.0,5.40,1.0,N,68,74,1,21.90,3.5,0.5,3.00,0.0,1.0,29.90,2.5,0.0
125,1,2024-05-01 00:34:06,2024-05-01 00:36:16,0.0,0.50,1.0,N,230,161,1,5.10,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0
206,1,2024-05-01 00:43:45,2024-05-01 00:50:50,0.0,1.60,1.0,N,246,48,1,9.30,3.5,0.5,2.85,0.0,1.0,17.15,2.5,0.0
325,1,2024-05-01 00:21:14,2024-05-01 00:27:13,0.0,2.20,1.0,N,229,262,1,10.70,3.5,0.5,3.10,0.0,1.0,18.80,2.5,0.0
398,1,2024-05-01 00:35:14,2024-05-01 00:40:57,0.0,0.70,1.0,N,114,249,1,7.20,3.5,0.5,2.40,0.0,1.0,14.60,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3723828,2,2024-05-31 23:45:00,2024-06-01 00:09:00,NaN,4.07,NaN,None,88,186,0,29.93,0.0,0.5,0.00,0.0,1.0,33.93,NaN,NaN
3723829,2,2024-05-31 23:10:52,2024-05-31 23:38:05,NaN,4.81,NaN,None,231,143,0,30.71,0.0,0.5,0.00,0.0,1.0,34.71,NaN,NaN
3723830,2,2024-05-31 23:23:57,2024-05-31 23:38:35,NaN,2.03,NaN,None,142,140,0,17.15,0.0,0.5,4.23,0.0,1.0,25.38,NaN,NaN
3723831,2,2024-05-31 23:44:12,2024-05-31 23:47:44,NaN,0.94,NaN,None,141,263,0,-1.00,0.0,0.5,0.00,0.0,1.0,3.00,NaN,NaN


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
250,1,2024-05-01 00:43:06,2024-05-01 00:43:40,1.0,0.0,5.0,N,238,238,1,10.00,0.0,0.0,2.75,0.0,1.0,13.75,0.0,0.00
278,2,2024-05-01 00:42:36,2024-05-01 00:42:51,1.0,0.0,5.0,N,10,10,1,20.00,0.0,0.0,2.00,0.0,1.0,23.00,0.0,0.00
292,2,2024-05-01 00:29:54,2024-05-01 00:30:01,1.0,0.0,5.0,N,125,125,1,20.00,0.0,0.0,0.00,0.0,1.0,21.00,0.0,0.00
338,2,2024-05-01 00:12:05,2024-05-01 00:12:13,2.0,0.0,5.0,N,265,265,1,109.00,0.0,0.0,22.00,0.0,1.0,132.00,0.0,0.00
519,2,2024-05-01 00:14:39,2024-05-01 00:14:53,2.0,0.0,1.0,N,132,132,2,3.00,1.0,0.5,0.00,0.0,1.0,7.25,0.0,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3723538,2,2024-05-31 23:41:34,2024-05-31 23:54:50,NaN,0.0,NaN,None,148,232,0,10.86,0.0,0.5,0.00,0.0,1.0,14.86,NaN,NaN
3723548,1,2024-05-31 23:52:57,2024-06-01 00:00:00,NaN,0.0,NaN,None,234,107,0,14.47,0.0,0.5,0.00,0.0,1.0,18.47,NaN,NaN
3723648,2,2024-05-31 23:33:53,2024-06-01 00:04:55,NaN,0.0,NaN,None,68,263,0,25.57,0.0,0.5,0.00,0.0,1.0,29.57,NaN,NaN
3723725,1,2024-05-31 23:37:10,2024-05-31 23:37:11,NaN,0.0,NaN,None,144,144,0,28.27,0.0,0.5,0.00,0.0,1.0,32.27,NaN,NaN


In [6]:
dropped_passenger_trip = df[(df['passenger_count'].isin([0])) | (df['trip_distance']).isin([0])]
dropped_passenger_trip.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87679 entries, 21 to 3723774
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               87679 non-null  int32         
 1   tpep_pickup_datetime   87679 non-null  datetime64[us]
 2   tpep_dropoff_datetime  87679 non-null  datetime64[us]
 3   passenger_count        81408 non-null  float64       
 4   trip_distance          87679 non-null  float64       
 5   RatecodeID             81408 non-null  float64       
 6   store_and_fwd_flag     81408 non-null  object        
 7   PULocationID           87679 non-null  int32         
 8   DOLocationID           87679 non-null  int32         
 9   payment_type           87679 non-null  int64         
 10  fare_amount            87679 non-null  float64       
 11  extra                  87679 non-null  float64       
 12  mta_tax                87679 non-null  float64       
 13  tip

In [7]:
df = df[(df['passenger_count'] != 0) & (df['trip_distance'] != 0)]
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-05-01 00:59:15,2024-05-01 01:23:50,1.0,6.10,1.0,N,138,145,1,28.20,7.75,0.5,5.00,0.00,1.0,42.45,0.0,1.75
1,2,2024-04-30 23:58:26,2024-05-01 00:29:42,1.0,11.23,1.0,N,138,249,1,46.40,6.00,0.5,8.72,0.00,1.0,66.87,2.5,1.75
2,2,2024-05-01 00:57:17,2024-05-01 01:14:15,1.0,9.02,1.0,N,138,170,1,35.90,6.00,0.5,10.57,6.94,1.0,65.16,2.5,1.75
3,2,2024-05-01 00:24:47,2024-05-01 00:48:51,1.0,6.53,1.0,N,87,133,1,30.30,1.00,0.5,7.06,0.00,1.0,42.36,2.5,0.00
4,2,2024-05-01 00:11:20,2024-05-01 00:52:10,1.0,14.38,1.0,N,161,165,1,61.80,1.00,0.5,0.00,0.00,1.0,66.80,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3723828,2,2024-05-31 23:45:00,2024-06-01 00:09:00,NaN,4.07,NaN,None,88,186,0,29.93,0.00,0.5,0.00,0.00,1.0,33.93,NaN,NaN
3723829,2,2024-05-31 23:10:52,2024-05-31 23:38:05,NaN,4.81,NaN,None,231,143,0,30.71,0.00,0.5,0.00,0.00,1.0,34.71,NaN,NaN
3723830,2,2024-05-31 23:23:57,2024-05-31 23:38:35,NaN,2.03,NaN,None,142,140,0,17.15,0.00,0.5,4.23,0.00,1.0,25.38,NaN,NaN
3723831,2,2024-05-31 23:44:12,2024-05-31 23:47:44,NaN,0.94,NaN,None,141,263,0,-1.00,0.00,0.5,0.00,0.00,1.0,3.00,NaN,NaN


In [8]:
df.dropna(inplace=True)

In [9]:
df.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3319162,2,2024-05-31 23:35:15,2024-05-31 23:39:32,1.0,0.86,1.0,N,163,142,1,6.5,1.0,0.5,2.30,0.0,1.0,13.80,2.5,0.0
3319163,2,2024-05-31 23:52:36,2024-06-01 00:06:40,1.0,2.08,1.0,N,161,234,1,14.2,1.0,0.5,3.84,0.0,1.0,23.04,2.5,0.0
3319164,2,2024-05-31 23:41:11,2024-05-31 23:47:48,1.0,1.55,1.0,N,75,151,2,9.3,1.0,0.5,0.00,0.0,1.0,11.80,0.0,0.0
3319165,2,2024-05-31 23:01:30,2024-05-31 23:07:47,1.0,0.88,1.0,N,68,48,1,7.9,1.0,0.5,0.00,0.0,1.0,12.90,2.5,0.0
3319166,2,2024-05-31 23:21:48,2024-05-31 23:32:52,1.0,2.57,1.0,N,48,140,1,14.2,1.0,0.5,3.84,0.0,1.0,23.04,2.5,0.0


# Dimensional Data Modelling

## Vendor Dimensional Table

In [10]:
# Create Vendor Dimensional Table
vendor_dim = df[['VendorID']].drop_duplicates().reset_index(drop=True)

mapping = {
    1: "Creative Mobile Technologies, LLC",
    2: "VeriFone Inc."
}

vendor_dim['vendor_provider'] = vendor_dim['VendorID'].map(mapping)
vendor_dim = vendor_dim[['VendorID', 'vendor_provider']]
vendor_dim.head()

,VendorID,vendor_provider
0,1,"Creative Mobile Technologies, LLC"
1,2,VeriFone Inc.


## RatecodID Dimensional Table

In [29]:
# Create RatecodeId Dimensional Table
rate_code_dim = df[['RatecodeID']].drop_duplicates().reset_index(drop=True)
rate_code_dim.dropna(inplace=True)

mapping = {
    1: "Standard Rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride"
}

rate_code_dim['rate_code_type'] = rate_code_dim['RatecodeID'].map(mapping)

rate_code_dim = rate_code_dim[['RatecodeID', 'rate_code_type']]

rate_code_dim.head()

,RatecodeID,rate_code_type
0,1.0,Standard Rate
1,2.0,JFK
2,5.0,Negotiated fare
3,4.0,Nassau or Westchester
4,3.0,Newark


## Payment Type Dimensional

In [30]:
# Payment Type Dimensional 
payment_type_dim = df[['payment_type']].drop_duplicates().reset_index(drop=True)

mapping = {
    1: 'Credit card',
    2: 'Cash',
    3: 'No charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided trip'
}

payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(mapping)
payment_type_dim = payment_type_dim[['payment_type', 'payment_type_name']]
payment_type_dim

,payment_type,payment_type_name
0,1,Credit card
1,2,Cash
2,4,Dispute
3,3,No charge


## Pickup dan Dropoff Location Dimensional Table

In [13]:
# Locatiton Dimensional Table

pick_up_dim = df[['PULocationID']].drop_duplicates().reset_index(drop=True)
pick_up_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   PULocationID  251 non-null    int32
dtypes: int32(1)
memory usage: 1.1 KB


In [14]:
import pandas as pd

location_df = pd.read_csv('https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv')
location_df.columns = location_df.columns.str.lower()

In [15]:
location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   locationid    265 non-null    int64 
 1   borough       264 non-null    object
 2   zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [16]:
# Specify your transformation logic here
location_df = location_df.iloc[:-2]
location_df.columns = location_df.columns.str.lower()
location_df.rename(columns={'locationid': 'location_id'}, inplace=True)

In [17]:
# Join pick_up_dim dengan location_df menggunakan PULocationID dan location_id
pick_up_dim = pick_up_dim.merge(location_df, 
                                left_on='PULocationID', 
                                right_on='location_id', 
                                how='inner'
                )

In [31]:
# Tampilkan hasil
pick_up_dim = pick_up_dim[['PULocationID', 'borough', 'zone', 'service_zone']]
pick_up_dim

,PULocationID,borough,zone,service_zone
0,138,Queens,LaGuardia Airport,Airports
1,87,Manhattan,Financial District North,Yellow Zone
2,161,Manhattan,Midtown Center,Yellow Zone
3,186,Manhattan,Penn Station/Madison Sq West,Yellow Zone
4,237,Manhattan,Upper East Side South,Yellow Zone
...,...,...,...,...
244,206,Staten Island,Saint George/New Brighton,Boro Zone
245,118,Staten Island,Heartland Village/Todt Hill,Boro Zone
246,27,Queens,Breezy Point/Fort Tilden/Riis Beach,Boro Zone
247,172,Staten Island,New Dorp/Midland Beach,Boro Zone


In [19]:
# Locatiton Dimensional Table

drop_off_dim = df[['DOLocationID']].drop_duplicates().reset_index(drop=True)
drop_off_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   DOLocationID  259 non-null    int32
dtypes: int32(1)
memory usage: 1.1 KB


In [20]:
# Join pick_up_dim dengan location_df menggunakan PULocationID dan location_id
drop_off_dim = drop_off_dim.merge(location_df, 
                                left_on='DOLocationID', 
                                right_on='location_id', 
                                how='inner'
                )

In [32]:
# Tampilkan hasil
drop_off_dim = drop_off_dim[['DOLocationID', 'borough', 'zone', 'service_zone']]
drop_off_dim

,DOLocationID,borough,zone,service_zone
0,145,Queens,Long Island City/Hunters Point,Boro Zone
1,249,Manhattan,West Village,Yellow Zone
2,170,Manhattan,Murray Hill,Yellow Zone
3,133,Brooklyn,Kensington,Boro Zone
4,165,Brooklyn,Midwood,Boro Zone
...,...,...,...,...
252,109,Staten Island,Great Kills,Boro Zone
253,176,Staten Island,Oakwood,Boro Zone
254,204,Staten Island,Rossville/Woodrow,Boro Zone
255,84,Staten Island,Eltingville/Annadale/Prince's Bay,Boro Zone


In [22]:
df = df.reset_index(drop=True)

In [23]:
df['trip_id'] = df.index

In [24]:
df = df[['trip_id', 'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']]
df.head()

,trip_id,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,0,1,2024-05-01 00:59:15,2024-05-01 01:23:50,1.0,6.10,1.0,N,138,145,1,28.2,7.75,0.5,5.00,0.00,1.0,42.45,0.0,1.75
1,1,2,2024-04-30 23:58:26,2024-05-01 00:29:42,1.0,11.23,1.0,N,138,249,1,46.4,6.00,0.5,8.72,0.00,1.0,66.87,2.5,1.75
2,2,2,2024-05-01 00:57:17,2024-05-01 01:14:15,1.0,9.02,1.0,N,138,170,1,35.9,6.00,0.5,10.57,6.94,1.0,65.16,2.5,1.75
3,3,2,2024-05-01 00:24:47,2024-05-01 00:48:51,1.0,6.53,1.0,N,87,133,1,30.3,1.00,0.5,7.06,0.00,1.0,42.36,2.5,0.00
4,4,2,2024-05-01 00:11:20,2024-05-01 00:52:10,1.0,14.38,1.0,N,161,165,1,61.8,1.00,0.5,0.00,0.00,1.0,66.80,2.5,0.00


In [33]:
fact_table = df.merge(vendor_dim, left_on='VendorID', right_on='VendorID')\
                .merge(rate_code_dim, left_on='RatecodeID', right_on='RatecodeID')\
                .merge(pick_up_dim, left_on='PULocationID', right_on='PULocationID')\
                .merge(drop_off_dim, left_on='DOLocationID', right_on='DOLocationID')\
                .merge(payment_type_dim, left_on='payment_type', right_on='payment_type')\
               [['trip_id', 'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']]

In [34]:
fact_table

,trip_id,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,0,1,2024-05-01 00:59:15,2024-05-01 01:23:50,1.0,6.10,1.0,N,138,145,1,28.2,7.75,0.5,5.00,0.00,1.0,42.45,0.0,1.75
1,1,2,2024-04-30 23:58:26,2024-05-01 00:29:42,1.0,11.23,1.0,N,138,249,1,46.4,6.00,0.5,8.72,0.00,1.0,66.87,2.5,1.75
2,2,2,2024-05-01 00:57:17,2024-05-01 01:14:15,1.0,9.02,1.0,N,138,170,1,35.9,6.00,0.5,10.57,6.94,1.0,65.16,2.5,1.75
3,3,2,2024-05-01 00:24:47,2024-05-01 00:48:51,1.0,6.53,1.0,N,87,133,1,30.3,1.00,0.5,7.06,0.00,1.0,42.36,2.5,0.00
4,4,2,2024-05-01 00:11:20,2024-05-01 00:52:10,1.0,14.38,1.0,N,161,165,1,61.8,1.00,0.5,0.00,0.00,1.0,66.80,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3204961,3237754,2,2024-05-31 23:35:15,2024-05-31 23:39:32,1.0,0.86,1.0,N,163,142,1,6.5,1.00,0.5,2.30,0.00,1.0,13.80,2.5,0.00
3204962,3237755,2,2024-05-31 23:52:36,2024-06-01 00:06:40,1.0,2.08,1.0,N,161,234,1,14.2,1.00,0.5,3.84,0.00,1.0,23.04,2.5,0.00
3204963,3237756,2,2024-05-31 23:41:11,2024-05-31 23:47:48,1.0,1.55,1.0,N,75,151,2,9.3,1.00,0.5,0.00,0.00,1.0,11.80,0.0,0.00
3204964,3237757,2,2024-05-31 23:01:30,2024-05-31 23:07:47,1.0,0.88,1.0,N,68,48,1,7.9,1.00,0.5,0.00,0.00,1.0,12.90,2.5,0.00
